In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json


#Loading data
columns_name = ['id', 'date', 'latitude', 'longitude' ]
df = pd.read_csv('../Data_Anom/real_data_set.csv', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")

df['week'] = df['date'].dt.isocalendar().week

In [2]:
id_week_ori = df.groupby('id')['week'].unique()
list_id_ori = df["id"].unique()
print(np.sort(list_id_ori), len(list_id_ori))

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110] 69


In [3]:
for week, group in df.groupby('week'):
    print(f'Week {week}:')
    df_week = df[df['week'] == week]
    count_id_ori_week = df_week['id'].value_counts().reset_index()
    count_id_ori_week.columns = ['id', 'count']
    count_id_ori_week = count_id_ori_week.sort_values(by='count', ascending=True)
    list_count_id_ori_week = count_id_ori_week['id'].tolist()
    print(list_count_id_ori_week, len(list_count_id_ori_week))
    print()

Week 10:
[98, 18, 63, 107, 5, 66, 78, 32, 30, 77, 68, 29, 48, 110, 35, 2, 13, 83, 73, 60, 67, 25, 71, 16, 34, 24, 49, 1, 39, 41, 21, 89, 23, 72, 43, 54, 59, 26, 9, 28, 17, 37, 75, 58, 44, 15, 51, 31, 8, 14, 7, 52, 50, 27, 55, 87, 36, 11, 69, 42, 4, 38, 62, 6] 64

Week 11:
[29, 98, 5, 48, 58, 30, 77, 107, 110, 21, 78, 63, 68, 54, 25, 49, 37, 6, 1, 75, 60, 16, 13, 23, 34, 67, 89, 2, 83, 14, 41, 15, 24, 11, 52, 36, 9, 26, 44, 59, 73, 43, 8, 31, 28, 7, 17, 55, 72, 62, 87, 4, 38, 51, 50, 69, 27, 42, 53] 59

Week 12:
[34, 63, 38, 84, 65, 16, 55, 71, 78, 68, 70, 15, 77, 30, 107, 1, 9, 24, 44, 18, 32, 53, 6, 81, 66, 41, 52, 8, 2, 60, 75, 49, 26, 72, 67, 69, 43, 42, 51, 62, 31, 59, 13, 73, 28, 50, 87, 7, 17, 27, 4] 51

Week 13:
[44, 71, 77, 29, 70, 9, 84, 43, 54, 18, 41, 63, 49, 35, 52, 60, 24, 68, 83, 30, 32, 78, 66, 1, 51, 65, 17, 75, 67, 2, 8, 13, 28, 59, 72, 31, 73, 81, 62, 69, 7, 26, 27, 42, 53, 6, 50, 4] 48

Week 14:
[71, 17, 52, 78, 9, 84, 29, 35, 54, 70, 68, 53, 27, 32, 63, 66, 49, 75, 

In [4]:
for week, group in df.groupby('week'):
    print(f'Week {week}:')
    df_week = df[df['week'] == week]    
    count_id_ori_each_week = df_week['id'].value_counts().reset_index()
    count_id_ori_each_week.columns = ['id', 'count']

    print(count_id_ori_each_week)

Week 10:
     id   count
0     6  238005
1    62  209941
2    38  200805
3     4  198951
4    42  183845
..  ...     ...
59    5    4110
60  107    3215
61   63    1221
62   18     923
63   98     444

[64 rows x 2 columns]
Week 11:
     id   count
0    53  311171
1    42  288453
2    27  256638
3    69  243740
4    50  241614
5    51  216169
6    38  208422
7     4  195647
8    87  195320
9    62  190390
10   72  163536
11   55  159087
12   17  149269
13    7  140274
14   28  137035
15   31  136774
16    8  136629
17   43  134535
18   73  122894
19   59  121514
20   44  116557
21   26  113238
22    9  109638
23   36   91063
24   52   84146
25   11   76972
26   24   74595
27   15   67135
28   41   62907
29   14   57700
30   83   57338
31    2   56993
32   89   54949
33   67   53496
34   34   51920
35   23   40703
36   13   39178
37   16   38757
38   60   35893
39   75   35739
40    1   32587
41    6   29598
42   37   26438
43   49   23574
44   25   20949
45   54   19377
46   68   16790

In [3]:
#Random minute : returns a new date by adding or subtracting the random number of minutes, depending on whether the hour remains the same.
def random_minute(date):
    rand_min = random.randint(-29,29)
    return date - dt.timedelta(minutes=rand_min) if date.hour == (date - dt.timedelta(minutes=rand_min)).hour else date + dt.timedelta(minutes=rand_min)

#Change date to random (same week)
def change_date(date):
    rand_day = int(np.random.choice([-1,0,1], p=[0.02, 0.96, 0.02]))
    return date - dt.timedelta(days=rand_day) if date.isocalendar()[1] == (date - dt.timedelta(days=rand_day)).isocalendar()[1] else date + dt.timedelta(days=rand_day)

In [4]:
#Set id to 'DEL' with lines duplicated and retain only the first occurrence of each unique combination of values in these columns.
id_time = df.drop_duplicates(['id', 'date', 'latitude', 'longitude'])
print(id_time.shape)

index_not_change = np.array(id_time.index)
df.loc[~df.index.isin(index_not_change) , 'id'] = "DEL"

print(df.head())

(16654892, 5)
    id                date  latitude  longitude  week
0    1 2015-03-04 00:35:00  4.870147  45.772140    10
1    1 2015-03-04 00:35:00  4.870218  45.772095    10
2    1 2015-03-04 00:35:00  4.870210  45.772072    10
3  DEL 2015-03-04 00:35:00  4.870210  45.772072    10
4  DEL 2015-03-04 00:35:00  4.870210  45.772072    10


In [5]:
df

,id,date,latitude,longitude,week
0,1,2015-03-04 00:35:00,4.870147,45.772140,10
1,1,2015-03-04 00:35:00,4.870218,45.772095,10
2,1,2015-03-04 00:35:00,4.870210,45.772072,10
3,DEL,2015-03-04 00:35:00,4.870210,45.772072,10
4,DEL,2015-03-04 00:35:00,4.870210,45.772072,10
...,...,...,...,...,...
34551844,110,2015-03-12 16:23:00,2.343094,48.891650,11
34551845,110,2015-03-12 16:23:00,2.343094,48.891650,11
34551846,110,2015-03-12 16:23:00,2.343094,48.891649,11
34551847,110,2015-03-12 16:23:00,2.343094,48.891649,11


In [6]:
#Change lat-long
df['latitude'] = df['latitude'] + 0.005 * (1 / (1 + np.exp(-np.sin(df['latitude']))))
df['longitude'] = df['longitude'] + 0.005 * (1 / (1 + np.exp(-np.sin(df['longitude']))))

In [7]:
#Apply random_minute for the ids not DEL
df.loc[df['id'] != 'DEL', 'date'] = df.loc[df['id'] != 'DEL', 'date'].apply(random_minute)

#Apply change_date for the ids not DEL
df.loc[df['id'] != 'DEL', 'date'] = df.loc[df['id'] != 'DEL', 'date'].apply(change_date) 

In [10]:
for week, group in id_time.groupby('week'):
    print(f'Week {week}:')
    id_time_week = id_time[id_time['week'] == week]    
    count_id_ori_each_week = id_time_week['id'].value_counts().reset_index()
    count_id_ori_each_week.columns = ['id', 'count']

    print(count_id_ori_each_week)

Week 10:
     id   count
0     6  222194
1     4  117768
2    11   97240
3    38   87014
4    42   66530
..  ...     ...
59   66    2992
60  107    2210
61   63     921
62   18     752
63   98     344

[64 rows x 2 columns]
Week 11:
     id   count
0    50  126067
1    69  121589
2    27  116479
3    51  113625
4     4  109532
5    42   99890
6    38   97228
7    53   93147
8    72   92751
9     8   77954
10   28   74121
11   55   67150
12    9   66603
13   43   62098
14    7   57972
15   26   52211
16   59   51345
17   17   49828
18   31   48240
19    2   44983
20   44   44204
21   11   41443
22   24   41126
23   87   40434
24   15   38426
25   52   38146
26   14   36644
27   89   32242
28   67   32093
29   73   30715
30   62   26404
31    6   26272
32   83   25460
33   23   24810
34   34   24373
35    1   22629
36   36   22407
37   13   22142
38   41   22126
39   75   20946
40   25   15693
41   60   13751
42   37   12962
43   68   12558
44   54   10233
45   63   10210
46   49    9690

In [8]:
#Id anonymisation
id_time['new_id'] =  3 * id_time['id'] + id_time['week']

id_time['new_id'] = id_time['new_id'].apply(lambda x: int(x * np.log(x) + (abs(1 - x)) * np.log(abs(1 - x))))
df.loc[df['id'] != 'DEL', 'id'] = id_time['new_id']

C:\Users\Phuoc Le\AppData\Local\Temp\ipykernel_15792\1271131018.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_time['new_id'] =  3 * id_time['id'] + id_time['week']
C:\Users\Phuoc Le\AppData\Local\Temp\ipykernel_15792\1271131018.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_time['new_id'] = id_time['new_id'].apply(lambda x: int(x * np.log(x) + (abs(1 - x)) * np.log(abs(1 - x))))


In [9]:
df

,id,date,latitude,longitude,week
0,63,2015-03-04 00:21:00,4.871504,45.775772,10
1,63,2015-03-04 00:22:00,4.871575,45.775727,10
2,63,2015-03-04 00:54:00,4.871567,45.775703,10
3,DEL,2015-03-04 00:35:00,4.871567,45.775703,10
4,DEL,2015-03-04 00:35:00,4.871567,45.775703,10
...,...,...,...,...,...
34551844,3970,2015-03-12 16:52:00,2.346453,48.893013,11
34551845,3970,2015-03-12 16:29:00,2.346453,48.893013,11
34551846,3970,2015-03-12 16:28:00,2.346453,48.893013,11
34551847,3970,2015-03-12 16:52:00,2.346453,48.893013,11


In [19]:
new_df = df.loc[df['id'] != 'DEL']

In [20]:
new_df

,id,date,latitude,longitude,week
0,63,2015-03-04 00:12:00,4.871504,45.775772,10
1,63,2015-03-04 00:27:00,4.871575,45.775727,10
2,63,2015-03-04 00:39:00,4.871567,45.775703,10
10,63,2015-03-04 00:48:00,4.871567,45.775703,10
29,63,2015-03-04 00:40:00,4.871474,45.775767,10
...,...,...,...,...,...
34551844,3970,2015-03-12 16:19:00,2.346453,48.893013,11
34551845,3970,2015-03-13 16:07:00,2.346453,48.893013,11
34551846,3970,2015-03-12 16:49:00,2.346453,48.893013,11
34551847,3970,2015-03-12 16:38:00,2.346453,48.893013,11


In [21]:
id_new_week_ori = new_df.groupby('id')['week'].unique()
list_new_id_ori = new_df["id"].unique()
print(np.sort(list_new_id_ori), len(list_new_id_ori))

[63 70 77 84 92 100 107 115 123 131 140 148 156 165 173 182 190 199 208
 217 226 235 244 253 262 271 281 290 299 309 318 328 347 376 386 396 405
 415 425 435 445 455 465 476 486 496 506 516 527 537 547 558 568 579 589
 600 610 621 631 685 695 717 728 738 749 760 771 782 793 804 815 826 837
 848 859 870 881 893 915 926 937 949 960 971 982 994 1005 1017 1028 1039
 1051 1062 1074 1085 1120 1131 1143 1154 1166 1189 1201 1212 1224 1283
 1294 1306 1318 1330 1342 1353 1365 1377 1389 1401 1413 1425 1437 1449
 1461 1473 1545 1557 1581 1593 1605 1617 1630 1642 1654 1666 1678 1691
 1703 1715 1727 1740 1752 1764 1776 1801 1813 1826 1912 1925 1950 1962
 1975 1987 2000 2012 2025 2037 2050 2062 2075 2087 2100 2113 2125 2138
 2150 2163 2176 2188 2201 2214 2226 2239 2252 2264 2277 2290 2303 2315
 2328 2341 2354 2366 2379 2392 2405 2418 2430 2443 2456 2469 2482 2495
 2507 2520 2533 2546 2559 2572 2585 2598 2611 2624 2637 2650 2663 2676
 2689 2702 2715 2728 2741 2754 2767 2780 2819 2832 2845 2858 2871 28

In [22]:
for week, group in new_df.groupby('week'):
    print(f'Week {week}:')
    new_df_week = new_df[new_df['week'] == week]    
    count_id_ori_each_week = new_df_week['id'].value_counts().reset_index()
    count_id_ori_each_week.columns = ['id', 'count']

    print(count_id_ori_each_week)

Week 10:
      id   count
0    182  222194
1    131  117768
2    318   97240
3   1189   87014
4   1330   66530
..   ...     ...
59  2214    2992
60  3834    2210
61  2100     921
62   527     752
63  3469     344

[64 rows x 2 columns]
Week 11:
      id   count
0   1630  126067
1   2341  121589
2    826  116479
3   1666  113625
4    140  109532
5   1342   99890
6   1201   97228
7   1740   93147
8   2456   92751
9    244   77954
10   859   74121
11  1813   67150
12   271   66603
13  1377   62098
14   217   57972
15   793   52211
16  1962   51345
17   506   49828
18   960   48240
19    92   44983
20  1413   44204
21   328   41443
22   728   41126
23  3042   40434
24   445   38426
25  1703   38146
26   415   36644
27  3122   32242
28  2264   32093
29  2495   30715
30  2075   26404
31   190   26272
32  2884   25460
33   695   24810
34  1062   24373
35    70   22629
36  1131   22407
37   386   22142
38  1306   22126
39  2572   20946
40   760   15693
41  2000   13751
42  1166   12962
43  230

In [10]:
df = df.drop('week', axis = 1)
df

,id,date,latitude,longitude
0,63,2015-03-04 00:21:00,4.871504,45.775772
1,63,2015-03-04 00:22:00,4.871575,45.775727
2,63,2015-03-04 00:54:00,4.871567,45.775703
3,DEL,2015-03-04 00:35:00,4.871567,45.775703
4,DEL,2015-03-04 00:35:00,4.871567,45.775703
...,...,...,...,...
34551844,3970,2015-03-12 16:52:00,2.346453,48.893013
34551845,3970,2015-03-12 16:29:00,2.346453,48.893013
34551846,3970,2015-03-12 16:28:00,2.346453,48.893013
34551847,3970,2015-03-12 16:52:00,2.346453,48.893013


In [11]:
# Save file
df.to_csv('logarithm.csv', sep='\t', index = False, header = False, columns = None)